# 2016 Bot 

### Overview
- Introduction 
- Data ETL
- Data Augmentation 
- Networks Architecture 
- Training 
- Model Averaging 
- Conclusion 

In [1]:
import os, sys, glob
import numpy as np
import cv2
from skimage.io import imread

from sklearn.cross_validation import KFold
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

In [2]:
# Data Augmentation 
from keras.preprocessing.image import ImageDataGenerator

# model build 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

from keras.utils import np_utils
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import model_from_json


Using Theano backend.
DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/kentc/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-2.7.12-64/tmpr43owg/265abc51f7c376c224983485238ff1a5.lib and object C:/Users/kentc/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-2.7.12-64/tmpr43owg/265abc51f7c376c224983485238ff1a5.exp

Using gpu device 0: GeForce GTX 970M (CNMeM is disabled, cuDNN 5103)
C:\Users\kentc\Anaconda2\lib\site-packages\theano\sandbox\cuda\__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [4]:
class Conf:
    def __init__(self, confPath):
        # load and store the configuration and update the object's dictionary
        conf = json.loads(open(confPath).read())
        self.__dict__.update(conf)

    def __getitem__(self, k):
        # return the value associated with the supplied key
        return self.__dict__.get(k, None)
    
    
def auto_resized(img,size):
    '''size = (width,height)'''
    size = tuple(size)
    resize_img = cv2.resize(img, size, interpolation=cv2.INTER_LINEAR)
    return resize_img

def TrainFilePath(folderPath, constrain=None, **kargs):
    '''
    (1) Output filepath and calssName
    (2) folderPath 
          --label_1
           -- xxx.jpg
    '''
    assert folderPath[-1]!='/'
    if constrain is None:
        constrain = ('avi', 'mp4','png','jpg') 
    for (rootDir, dirNames, fileNames) in os.walk(folderPath):
        for fileName in fileNames:
            if fileName.split('.')[-1] in constrain:
                yield (os.path.join(rootDir, fileName)) 
                
#img_channels = 3
def genTrX(filePath, resolution, img_channels=3):
    assert type(resolution) == tuple
    img = auto_resized(imread(filePath),resolution)  #conf['sliding_size']
    if img_channels==1:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    elif img_channels==3:
        img = img[:,:,:3]
    return img
                
def load_training(folderList, img_rows, img_cols, img_channels):
    TrY = []
    TrX = []
    TrY_template = np.eye(len(folderList))
    for eyeId, folderPath in enumerate(folderList):
        for imgPath in TrainFilePath(folderPath) :
            TrY.append(TrY_template[eyeId])
            TrX.append(genTrX(imgPath, (img_rows,img_cols), img_channels))
    print (len(TrX))
    return TrX, TrY

def create_folderList(rootDir):
    result=[]
    for a in os.listdir(rootDir):
        a = os.path.join(rootDir, a)
        if os.path.isdir(a):
            result.append(a) 
    return result


def reshapeShuffle(TrX, TrY, img_rows, img_cols, img_channels):
    trainX = np.asarray(TrX, dtype = np.uint8)
    trainX = trainX.reshape(trainX.shape[0], img_channels, img_rows, img_cols)
    trainX = trainX.astype('float32')
    trainY = np.asarray(TrY, dtype = np.float32)
    # shuffle
    trainX , trainY = shuffle(trainX,trainY)
    print ('Train_X : ',trainX.shape,'Train_Y' ,trainY.shape)
    return trainX , trainY

In [5]:
ROOT_Dir = 'D:\\2016bot_cv'

img_rows= 64

img_cols= 64

img_channels=3

folderList = create_folderList(ROOT_Dir)

Train_X, Train_Y = load_training(folderList, img_rows, img_cols, img_channels)



#TrainFilePath('D:\2015')

116481


In [6]:
train_X , train_Y = reshapeShuffle(Train_X, Train_Y, img_rows, img_cols, img_channels=img_channels)

('Train_X : ', (116481L, 3L, 64L, 64L), 'Train_Y', (116481L, 12L))


In [7]:
gen_Img = ImageDataGenerator(featurewise_center=True,
    samplewise_center=False,
    featurewise_std_normalization=True,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=20.,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None)



In [8]:
gen_Img.fit(train_X)

In [9]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD

def VGG_19(img_rows,img_cols,weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,img_rows,img_cols)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(5600, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5600, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(550, activation='softmax'))
    model.add(Dense(12, activation='softmax'))   
    if weights_path:
        model.load_weights(weights_path)

    return model

In [25]:
def simpleVGG(img_rows,img_cols,weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,img_rows,img_cols)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(800, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(800, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(250, activation='softmax'))
    model.add(Dense(12, activation='softmax'))   
    if weights_path:
        model.load_weights(weights_path)

    return model

In [31]:
def easyVGG(img_rows,img_cols,weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,img_rows,img_cols)))
    model.add(Convolution2D(32, 3, 3))
    model.add(MaxPooling2D((3,3), strides=(2,2)))

    model.add(Convolution2D(64, 3, 3))
    model.add(MaxPooling2D((3,3), strides=(2,2)))
    
    model.add(Flatten())
    model.add(Dense(1500, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1000, activation='tanh'))
    model.add(Dense(200, activation='softmax'))
    model.add(Dense(12, activation='softmax'))   
    if weights_path:
        model.load_weights(weights_path)

    return model

In [32]:

#model = VGG_19(img_rows,img_cols)
model = easyVGG(img_rows,img_cols)
#https://gist.github.com/baraldilorenzo/8d096f48a1be4a2d660d
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
zeropadding2d_39 (ZeroPadding2D) (None, 3, 66, 66)     0           zeropadding2d_input_10[0][0]     
____________________________________________________________________________________________________
convolution2d_42 (Convolution2D) (None, 32, 64, 64)    896         zeropadding2d_39[0][0]           
____________________________________________________________________________________________________
maxpooling2d_19 (MaxPooling2D)   (None, 32, 31, 31)    0           convolution2d_42[0][0]           
____________________________________________________________________________________________________
convolution2d_43 (Convolution2D) (None, 64, 29, 29)    18496       maxpooling2d_19[0][0]            
___________________________________________________________________________________________

In [33]:

kf = KFold(len(train_Y), n_folds=5)
for train, test in kf:
    #print (train, test)
    Tr_X = train_X[train]
    Te_X = train_X[test]
    Tr_Y = train_Y[train]
    Te_Y = train_Y[test]
    # fits the model on batches with real-time data augmentation:
    model.fit_generator(gen_Img.flow(Tr_X, Tr_Y, batch_size=62),
                    samples_per_epoch=len(Tr_X), nb_epoch=50, validation_data=(Te_X, Te_Y))



Epoch 1/50
93184/93184 [==============================] - 283s - loss: 2.4717 - acc: 0.1193 - val_loss: 2.4753 - val_acc: 0.1023
Epoch 2/50
93184/93184 [==============================] - 261s - loss: 2.4555 - acc: 0.1371 - val_loss: 2.4804 - val_acc: 0.1026
Epoch 3/50
93184/93184 [==============================] - 263s - loss: 2.4393 - acc: 0.1432 - val_loss: 2.4940 - val_acc: 0.1027
Epoch 4/50
93184/93184 [==============================] - 263s - loss: 2.4109 - acc: 0.1468 - val_loss: 2.5168 - val_acc: 0.1036
Epoch 5/50
93184/93184 [==============================] - 261s - loss: 2.3892 - acc: 0.1486 - val_loss: 2.5512 - val_acc: 0.1024
Epoch 6/50
93184/93184 [==============================] - 262s - loss: 2.3725 - acc: 0.1503 - val_loss: 2.4599 - val_acc: 0.1256
Epoch 7/50
93184/93184 [==============================] - 260s - loss: 2.3600 - acc: 0.1503 - val_loss: 2.5715 - val_acc: 0.1084
Epoch 8/50
93184/93184 [==============================] - 258s - loss: 2.3468 - acc: 0.1506 - val

ValueError: I/O operation on closed file

In [34]:
for x in range(100,1000,100):
    print (model.predict(Te_X[x].reshape(1,img_channels,img_rows,img_rows)) ,'--><--', np.argmax(Te_Y[x]) )
    print (model.predict_classes(Te_X[x].reshape(1,img_channels,img_rows,img_rows)) ,'--><--', np.argmax(Te_Y[x]) )

(array([[ 0.06765255,  0.05522573,  0.166471  ,  0.0549007 ,  0.23201817,
         0.07208199,  0.02534884,  0.0910301 ,  0.09874104,  0.04685798,
         0.05006972,  0.03960211]]), '--><--', 6)
1/1 [==============================] - 0s
(array([4], dtype=int64), '--><--', 6)
(array([[ 0.14910103,  0.01401548,  0.18644281,  0.00969182,  0.00395913,
         0.57948172,  0.00097937,  0.0009457 ,  0.0021289 ,  0.02555787,
         0.0246339 ,  0.00306222]]), '--><--', 5)
1/1 [==============================] - 0s
(array([5], dtype=int64), '--><--', 5)
(array([[ 0.14858544,  0.0128876 ,  0.18008217,  0.00885729,  0.00325871,
         0.59287524,  0.00084462,  0.00077624,  0.00179375,  0.02414658,
         0.02314821,  0.00274419]]), '--><--', 2)
1/1 [==============================] - 0s
(array([5], dtype=int64), '--><--', 2)
(array([[ 0.07450308,  0.05318019,  0.19474643,  0.04515579,  0.22066508,
         0.12756181,  0.01942939,  0.06375183,  0.07406041,  0.04869528,
         0.05303335

In [35]:
model_name='2016bot_0001'
model.save_weights('../hub/model/{}.h5'.format(model_name))
print ('saving model weight as ' + '../hub/model/{}.h5'.format(model_name))



# serialize model to JSON
model_json = model.to_json()
with open("../hub/model/{}.json".format(model_name), "w") as json_file:
    json_file.write(model_json)

print ('saving model struct as ' + "../hub/model/{}.json".format(model_name))

saving model weight as ../hub/model/2016bot_0001.h5
saving model struct as ../hub/model/2016bot_0001.json


In [ ]:
import pickle

output = open('X_62.pkl', 'wb')

# Pickle dictionary using protocol 0.
pickle.dump(train_X, output)
output.close()

output2 = open('Y_62.pkl', 'wb')
pickle.dump(train_X, output2)
output2.close()
# Pickle the list using the highest protocol available.
# pickle.dump(selfref_list, output, -1)

# output.close()

In [ ]:
score = model.evaluate(Te_X, Te_Y, verbose=0)


In [ ]:
# Clean the GPU memory
import gc
gc.collect()

In [ ]:
model.predict_proba(Te_X[3].reshape(1,img_channels,img_rows,img_rows))

In [ ]:

class ConvNetFactory:
	def __init__(self):
		pass

	@staticmethod
	def build(name, *args, **kargs):
		# define the network (i.e., string => function) mappings
		mappings = {
			"shallownet": ConvNetFactory.ShallowNet,
			"lenet": ConvNetFactory.LeNet,
			"karpathynet": ConvNetFactory.KarpathyNet,
			"minivggnet": ConvNetFactory.MiniVGGNet}

		# grab the builder function from the mappings dictionary
		builder = mappings.get(name, None)

		# if the builder is None, then there is not a function that can be used
		# to build to the network, so return None
		if builder is None:
			return None

		# otherwise, build the network architecture
		return builder(*args, **kargs)

	@staticmethod
	def ShallowNet(numChannels, imgRows, imgCols, numClasses, **kwargs):
		# initialzie the model
		model = Sequential()

		# define the first (and only) CONV => RELU layer
		model.add(Convolution2D(32, 3, 3, border_mode="same",
			input_shape=(numChannels, imgRows, imgCols)))
		model.add(Activation("relu"))

		# add a FC layer followed by the soft-max classifier
		model.add(Flatten())
		model.add(Dense(numClasses))
		model.add(Activation("softmax"))

		# return the network architecture
		return model

	@staticmethod
	def LeNet(numChannels, imgRows, imgCols, numClasses, activation="tanh", **kwargs):
		# initialize the model
		model = Sequential()

		# define the first set of CONV => ACTIVATION => POOL layers
		model.add(Convolution2D(20, 5, 5, border_mode="same",
			input_shape=(numChannels, imgRows, imgCols)))
		model.add(Activation(activation))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# define the second set of CONV => ACTIVATION => POOL layers
		model.add(Convolution2D(50, 5, 5, border_mode="same"))
		model.add(Activation(activation))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# define the first FC => ACTIVATION layers
		model.add(Flatten())
		model.add(Dense(500))
		model.add(Activation(activation))

		# define the second FC layer
		model.add(Dense(numClasses))

		# lastly, define the soft-max classifier
		model.add(Activation("softmax"))

		# return the network architecture
		return model

	@staticmethod
	def KarpathyNet():
		# initialize the model
		model = Sequential()

		# define the first set of CONV => RELU => POOL layers
		model.add(Convolution2D(16, 5, 5, border_mode="same",
			input_shape=(numChannels, imgRows, imgCols)))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# check to see if dropout should be applied to reduce overfitting
		if dropout:
			model.add(Dropout(0.25))

		# define the second set of CONV => RELU => POOL layers
		model.add(Convolution2D(20, 5, 5, border_mode="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# check to see if dropout should be applied to reduce overfitting
		if dropout:
			model.add(Dropout(0.25))

		# define the third set of CONV => RELU => POOL layers
		model.add(Convolution2D(20, 5, 5, border_mode="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# check to see if dropout should be applied to reduce overfitting
		if dropout:
			model.add(Dropout(0.5))

		# define the soft-max classifier
		model.add(Flatten())
		model.add(Dense(numClasses))
		model.add(Activation("softmax"))

		# return the network architecture
		return model

	@staticmethod
	def MiniVGGNet():
		# initialize the model
		model = Sequential()

		# define the first set of CONV => RELU => CONV => RELU => POOL layers
		model.add(Convolution2D(32, 3, 3, border_mode="same",
			input_shape=(numChannels, imgRows, imgCols)))
		model.add(Activation("relu"))
		model.add(Convolution2D(32, 3, 3))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2)))

		# check to see if dropout should be applied to reduce overfitting
		if dropout:
			model.add(Dropout(0.25))

		# define the second set of CONV => RELU => CONV => RELU => POOL layers
		model.add(Convolution2D(64, 3, 3, border_mode="same"))
		model.add(Activation("relu"))
		model.add(Convolution2D(64, 3, 3))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2)))

		# check to see if dropout should be applied to reduce overfitting
		if dropout:
			model.add(Dropout(0.25))

		# define the set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation("relu"))

		# check to see if dropout should be applied to reduce overfitting
		if dropout:
			model.add(Dropout(0.5))

		# define the soft-max classifier
		model.add(Dense(numClasses))
		model.add(Activation("softmax"))

		# return the network architecture
		return model